# Fingeprints DB ELT Script

In [69]:
import requests
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
import json

# pd.set_option('display.float_format', lambda x: '%.2f' % x) # suppress scientific notation
pd.set_option('display.max_rows', 150)

# Input Addresses and Desired Data

In [70]:
# add relevant addresses below
addresses = {}

addresses['FP Main'] = '0xbc49de68bcbd164574847a7ced47e7475179c76b'
addresses['FP OPEX'] = '0xc9044dd6162106edb5ece3af8a8e8b657b612e42'
addresses['HW Phase3'] = '0x45e77e6473676ab30ed64f3815fea7847d02d7b8'
addresses['HW Compensation'] = '0x7af805dcd0c18ed276f99d83197fe7a9d37b3c54'
addresses['HW Panels Competition'] = '0xf358e6b648579Bc43042B3cc7661dD5309EB06b3'


# don't mess with txtypes!
txtypes = {}

txtypes['normal'] = 'txlist'
txtypes['erc20'] = 'tokentx'
txtypes['nfts'] = 'tokennfttx'
txtypes['internal'] = 'txlistinternal'


# Get Last Block from Database

In [71]:
load_dotenv('.env')
DB_CREDS = os.environ.get('DB_CREDS')

# connect
conn = psycopg2.connect(DB_CREDS)
cur = conn.cursor()

# get last block by table/tx type
sql = {}
last_blocks = {}
for txtype in txtypes:
    sql[txtype] = f"select max(\"blockNumber\")+1 as block_number from financials.{txtype};"
    last_blocks[txtype] = pd.read_sql_query(sql[txtype], conn).iloc[0]['block_number']

# check whether addresses is new in each table
sql2 = {}
add_check = {}
for txtype in txtypes:
    add_check[txtype] = {}
    
    for address in addresses.values():
        add_check[txtype][address] = 0
        sql2[address] = f"select count(*) from financials.{txtype} where \"from\" = \'{address.lower()}\' or \"to\" = \'{address.lower()}\'"
        add_check[txtype][address] += pd.read_sql_query(sql2[address], conn).iloc[0]['count']


cur.close()

# Requests

In [72]:
ETHERSCAN_APIKEY = os.environ.get('ETHERSCAN_APIKEY')

dfs = {}
merge = []
# get data from etherscan api
for txtype in txtypes:
    for address in addresses.values():
        if add_check[txtype][address] == 0: # if address is new, start block = 0
            payload = {
                'module': 'account',
                'action': txtypes[txtype],
                'address': address,
                'startblock': '0',
                'endblock': '999999999',
                'sort': 'asc',
                'apikey': ETHERSCAN_APIKEY
            }
        else: # else, use last block from database
            payload = {
                'module': 'account',
                'action': txtypes[txtype],
                'address': address,
                'startblock': last_blocks[txtype],
                'endblock': '999999999',
                'sort': 'asc',
                'apikey': ETHERSCAN_APIKEY
            }

        r = requests.get('https://api.etherscan.io/api', params=payload)
        response = json.loads(r.text)['result']
        merge.append(pd.DataFrame.from_dict(response))

    dfs[txtype] = pd.concat(merge) # store all merged data in a df
    dfs[txtype] = dfs[txtype].drop_duplicates() # remove duplicates (txs between our own wallets)
    merge = [] # reset merge array to store new txtype data


# Save to CSVs

In [73]:
# ----------------------------------------------------------------------------------------
# export data to CSVs
for df in dfs:
    dfs[df].to_csv('outputs/' + df + '.csv', index=False)

addressdf = pd.DataFrame({'name': addresses.keys(),'address': addresses.values()})
addressdf.to_csv('outputs/addresses.csv', index=False)
# ----------------------------------------------------------------------------------------

# Send Data to Database

In [75]:
import psycopg2.extras as extras

# calculate how many rows of new data are to be inserted
for df in dfs:
    data_check = 0
    data_check += len(dfs[df].index)

    print('New rows:')
    if data_check == 0:
        print('no new data for: ' + df)
    else:
        print(df + ' - ' + str(len(dfs[df].index)))


if data_check != 0:
    # send data
    cursor = conn.cursor()
    for txtype in txtypes:
        # Create a list of tupples from the dataframe values
        tuples = [tuple(x) for x in dfs[txtype].to_numpy()]
        # Comma-separated dataframe columns
        cols = '\",\"'.join(list(dfs[txtype].columns))
        # SQL quert to execute
        query  = "INSERT INTO %s(\"%s\") VALUES %%s" % ('financials.'+ txtype, cols)
        # print(query)
        cursor = conn.cursor()
        try:
            extras.execute_values(cursor, query, tuples)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()

    print("Data uploaded to database.")
    cursor.close()
    conn.close()


New rows:
no new data for: normal
New rows:
no new data for: erc20
New rows:
no new data for: nfts
New rows:
no new data for: internal
